# Projet maison n° 3

In [1]:
# imports
import pandas as pd
import os
import re

## 1. US baby names

On va s'intéresser au dataset **National data** de la SSA : https://www.ssa.gov/oact/babynames/limits.html

1. Télécharger le dataset des prénoms US : https://www.ssa.gov/oact/babynames/names.zip

Lire la documentation associée.

2. Implémenter une fonction Python `df_names_us()` qui produit un unique DataFrame avec tous les fichiers en utilisant **pandas** (par ex. avec la fonction `pandas.concat()`), pas de bash :)

Ordre et noms des colonnes du DataFrame : "year", "name", "gender", "births"

Le DataFrame doit être trié selon l'année croissante puis selon l'ordre défini dans les différents fichiers (voir la documentation du dataset).

In [2]:
# names us
def df_names_us():
    fpath = "C:\\Users\\Abo\\Documents\\MS\\KitBigData\\Kit-Big-Data - Copie\\session3\\names\\" 
    folders = os.listdir(fpath)
    folders_list = []
    
    for sheet in folders:
        df = pd.read_csv(fpath + sheet, names=["name", "gender", "births"], header=None)
        df.insert(0, 'year', sheet[12:16]) 
        folders_list.append(df)
    df = pd.concat(folders_list, ignore_index=True) 
    
    return df

df = df_names_us()

df

,year,name,gender,births
0,,Mary,F,7065
1,,Anna,F,2604
2,,Emma,F,2003
3,,Elizabeth,F,1939
4,,Minnie,F,1746
...,...,...,...,...
2052776,,Zyeire,M,5
2052777,,Zyel,M,5
2052778,,Zyian,M,5
2052779,,Zylar,M,5


## 2. Prénoms français

On va s'intéresser au dataset **Fichiers France hors Mayotte** de l'INSEE :  https://www.insee.fr/fr/statistiques/2540004/

L'idée est de charger les données et ensuite de les conformer au DataFrame des prénoms US. Ainsi, toute manipulation sur le DataFrame des prénoms US pourra être directement réutilisée avec le DataFrame des prénoms français.
 
1. Télécharger le dataset des prénoms français : https://www.insee.fr/fr/statistiques/fichier/2540004/nat2021_csv.zip


Lire la documentation, ça peut être utile...
 
2. Implémenter une fonction `df_names_fr()` qui produit un DataFrame avec les prénoms français en prenant le DataFrame des prénoms US comme modèle :
 
 - Même ordre et mêmes noms des colonnes : "year", "name", "gender", "births"
 - Mêmes dtypes pour les colonnes
 - Mêmes valeurs pour la colonne "gender"
 - Seuls les prénoms de 2 caractères et plus sont conservés
 - La casse des prénoms doit être identique : initiales en majuscule, autres lettres en minuscule
 - Les lignes avec des données inutilisables doivent être supprimées
 - Les données sont triées à l'identique : year (↑), puis gender (↑), puis births (↓) et enfin name (↑)
 - L'index du DataFrame doit aller de 0 à N-1

In [3]:
# names fr
def df_names_fr():
    
    #Telechargement le dataset
    df = pd.read_csv('nat2021.csv',sep=';',names=['sexe', 'preusuel','annais','nombre'],dtype=object, keep_default_na=False).loc[1:]
    
    #Ordonnancement et renommage des colonnes
    df = df[['annais', 'preusuel', 'sexe', 'nombre']]
    df = df.rename(columns={'annais': 'year', 'preusuel': 'name', 'sexe':'gender', 'nombre':'births'})
    df = df.sort_values(by=['year', 'gender', 'births', 'name'], ascending=[True, True, False, True])
    
    df['gender'] = df['gender'].apply(lambda x: 'M' if (x==1) else 'F')
        
    df = df[df['name'].apply(lambda x : len(str(x))) > 1]  
    
    df = df[df['name'] != '_PRENOMS_RARES']
    df = df[df['year'] != 'XXXX'] 
    df['name'] = df['name'].apply(lambda x : str(x).title()) 

    df = df.sort_values(by=['year', 'gender', 'births', 'name'], ascending=[True, True, False, True])
    df.index = pd.RangeIndex(len(df))

    return df

    df = df_names_fr()
    
    df
   

## 3. Taux de change

On va s'intéresser au dataset des cours des devises de la Banque de France :  http://webstat.banque-france.fr/fr/browseBox.do?node=5385566

Les données sont dans le fichier "Webstat_Export.csv".

L'idée est de charger les données, de les nettoyer et de pouvoir accéder aux cours de certaines devises à partir de leur code ISO3. On retiendra uniquement les colonnes se terminant effectivement par un code ISO3 entre parenthèses. Par ex., "Dollar des Etats-Unis (USD)".

Implémenter une fonction `df_taux_change()` qui produit un DataFrame avec les taux de change par date pour une liste de codes ISO3 de devises passée en argument. L'index du DataFrame doit correspondre aux dates (voir la fonction `pd.to_datetime()` avec le format`"%d/%m/%Y"`) et doit être trié par ordre croissant. Les colonnes du DataFrame doivent correspondre aux devises sélectionnées.

In [4]:
# taux de change
def df_taux_change(devises):
    df = pd.DataFrame([], columns = devises)

    f_path = "C:\\Users\\Abo\\Documents\\MS\\KitBigData\\Kit-Big-Data - Copie\\session3\\Webstat_Export.csv"
    df = pd.read_csv(f_path, sep =';', skiprows=[i for i in range(1,6)])
    df = df.rename(columns={"Titre :":"Date :"})
    
    for i in df.columns.values:
        findall = re.findall('\((.*?)\)', i)
        if findall!=[]:
            df.rename(columns={str(i):str(findall[0])}, inplace = True)
        
    df['Date :'] = pd.to_datetime(df['Date :'], format='%d/%m/%Y')
    df = df.set_index('Date :')
    df = df.apply(lambda x: x.str.replace(',','.'))
    df = df.apply(lambda x: x.replace('-',None))
    df = df.astype(float)
    df.dropna(axis=0, inplace = True)
    df = df.sort_values(by=["Date :"], ascending=True)
    return df[devises]

devises = ["CHF", "GBP", "USD"]
df_taux_change(devises)

,CHF,GBP,USD
Date :,,,
2000-07-19,1.5473,0.61560,0.9216
2000-07-20,1.5488,0.61650,0.9220
2000-07-21,1.5511,0.61850,0.9363
2000-07-24,1.5555,0.61510,0.9332
2000-07-25,1.5538,0.62010,0.9410
...,...,...,...
2006-12-21,1.6032,0.67160,1.3178
2006-12-22,1.6029,0.67150,1.3192
2006-12-27,1.6058,0.67085,1.3159


### Tests

In [5]:
import unittest

class Session3Tests(unittest.TestCase):
    def test_df_names_us(self):
        df = df_names_us()
        # colonnes
        self.assertEqual(list(df.columns), ["year", "name", "gender", "births"])
        # lignes
        self.assertEqual(len(df), 2052781)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)
        
    def test_df_names_fr(self):
        df = df_names_fr()
        # colonnes
        self.assertEqual(list(df.columns), ["year", "name", "gender", "births"])
        # lignes
        self.assertEqual(len(df), 648330)
        # index
        self.assertTrue(isinstance(df.index, pd.RangeIndex))
        # test names
        self.assertTrue(df.loc[df.name.str.contains(r"^[A-Z]+$")].empty)
        self.assertTrue(df.loc[df.name.str.contains(r"-[a-z]+$")].empty)
        # test gender
        self.assertEqual(len(df), len(df.loc[df.gender=="F"]) + len(df.loc[df.gender=='M']))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)

    def test_df_taux_change(self):
        df = df_taux_change(["CHF", "GBP", "USD"])
        # colonnes
        self.assertEqual(list(df.columns), ["CHF", "GBP", "USD"])
        # index
        self.assertTrue(isinstance(df.index, pd.DatetimeIndex))
        # index trié
        self.assertEqual(list(df.index.argsort()), list(range(len(df))))
        # types taux
        self.assertTrue((df.dtypes == float).all())
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)

In [6]:
# run tests
def run_tests():
    test_suite = unittest.makeSuite(Session3Tests)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)

In [7]:
# run tests
run_tests()

test_df_names_fr (__main__.Session3Tests) ... ok
test_df_names_us (__main__.Session3Tests) ... ok
test_df_taux_change (__main__.Session3Tests) ... ok

----------------------------------------------------------------------
Ran 3 tests in 8.646s

OK
